In [1]:
import sqlite3 as sq
import pandas as pd

In [2]:
con = sq.connect("NPOPlayer.db")
def run_query(con, table, query):
    cur = con.cursor()
    cur.execute(query)
    results = cur.fetchall()
    return results

def convert_db_table_to_DF(con, table):
    header_query = "SELECT name FROM pragma_table_info('" + table + "') ORDER BY cid;"
    cols_init = run_query(con, table, header_query)
    cols = [cols_init[i][0] for i in range(len(cols_init))]
    content_query = "SELECT * from " + table
    data = run_query(con, table, content_query)
    df = pd.DataFrame(data, columns= cols)
    return df

In [4]:
df = convert_db_table_to_DF(con, "NPOPLAYER")

In [20]:
import spacy
from tqdm import tqdm

# use displacy to visually show the entities. 
from spacy import displacy

# load spacy model. Alternatively you can use en_core_web_lg
nlp = spacy.load("nl_core_news_sm")

In [90]:
df_sub = df.copy()
#df_sub = df.sample(6000).copy()

In [ ]:
ruler = nlp.add_pipe('entity_ruler', before='ner')

patterns = [
  {"label": "PRODUCT", "pattern": "Wedstrijdanalyse"}
]
ruler.add_patterns(patterns)

In [91]:
def process(x):
  # there are some pesky NaN in the data. Easy but not so elegant way to fix this.
  if pd.isna(x) == False:
    doc = nlp(x)
  else:
    doc = ''
  return doc


tqdm.pandas()
df_sub['plot_entities'] = df_sub['longSummary'].progress_apply(lambda x: process(x))


100%|██████████| 132013/132013 [21:18<00:00, 103.27it/s]


In [ ]:
df_sub.to_pickle("NER.pkl")

In [89]:
def get_events(x):
  events = []
  if x != '':
    for entities in x.ents:
      if entities.label_ == 'EVENT':
        events.append(entities.text)
      elif entities.label_ == 'LAW':
        events.append(entities.text)
      elif entities.label_ == 'LOC':
        events.append(entities.text)
      elif entities.label_ == 'GPE':
        events.append(entities.text)
      elif entities.label_ == 'PERSON':
        events.append(entities.text)
      elif entities.label_ == 'PRODUCT':
        events.append(entities.text)
      elif entities.label_ == 'WORK_OF_ART':
        events.append(entities.text)
  return events

# code goes here
df_sub['tags'] = df_sub['plot_entities'].progress_apply(lambda x: get_events(x))
len(df_sub[df_sub["tags"].str.len() == 0])

100%|██████████| 6000/6000 [00:00<00:00, 66619.96it/s]


,mediaID,mainTitle,broadcaster,longSummary,shortSummary,icons,subTitle,thumbnail,kids,shareLink,plot_entities,events,tags
81684,POW_00263825,NOS Jeugdjournaal,NOS,,,,28 dec 2010 18:45 - Seizoen 13 Afl. 304 - NOS ...,https://images.npo.nl/header/2560x1440/333970.jpg,false,https://www.npostart.nl/POW_00263825,(),[],[]
3548,NPS_1237999,Andere Tijden,VPRO,,,,20 mrt 2014 21:25 - Seizoen 24 Afl. 11 - Waken...,https://images.npo.nl/header/2560x1440/549850.jpg,false,https://www.npostart.nl/NPS_1237999,(),[],[]
114297,POW_02758940,Tijd voor MAX,MAX,Compilatie van fragmenten uit de uitzendingen ...,Compilatie van fragmenten van de afgelopen week.,leeftijd-al,27 sep 2015 08:30 - Seizoen 3 Afl. 4 - Compila...,https://images.npo.nl/header/2560x1440/510499.jpg,false,https://www.npostart.nl/POW_02758940,"(Compilatie, van, fragmenten, uit, de, uitzend...",[],[]
108633,AT_2119814,Sterren NL Top 25,AVROTROS,Overzicht van de populairste Nederlandstalige ...,Overzicht van de populairste Nederlandstalige ...,leeftijd-al,19 okt 2019 12:20 - Seizoen 14 Afl. 42 - Sterr...,https://images.npo.nl/header/2560x1440/1288537...,false,https://www.npostart.nl/AT_2119814,"(Overzicht, van, de, populairste, Nederlandsta...",[],[]
78785,POW_00344693,NOS Jeugdjournaal,NOS,,,,15 sep 2011 18:45 - Seizoen 15 Afl. 256 - NOS ...,https://images.npo.nl/header/2560x1440/369005.jpg,false,https://www.npostart.nl/POW_00344693,(),[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23815,VARA_101157294,De Wereld Draait Door,VARA,Liveprogramma met gasten uit de wereld van nie...,Liveprogramma met tafelheer Marc-Marie Huijbre...,,9 nov 2007 19:30 - Seizoen 3 Afl. 152 - De we...,https://images.poms.omroep.nl/image/s2560/c256...,false,https://www.npostart.nl/VARA_101157294,"(Liveprogramma, met, gasten, uit, de, wereld, ...",[],[]
76326,POW_00195530,NOS Jeugdjournaal,NOS,,,,5 apr 2009 18:45 - Seizoen 10 Afl. 94 - NOS J...,https://images.poms.omroep.nl/image/s2560/c256...,false,https://www.npostart.nl/POW_00195530,(),[],[]
3304,VPWON_1236044,Andere Tijden,NTR,Ze dachten dat de oorlog over was. Even na Dol...,Ze dachten dat de oorlog over was. Even na Dol...,,7 apr 2015 21:20 - Seizoen 27 Afl. 12 - Verra...,https://images.npo.nl/header/2560x1440/779384.jpg,false,https://www.npostart.nl/VPWON_1236044,"(Ze, dachten, dat, de, oorlog, over, was, ., E...",[],[]
58623,VPWON_1272823,Koek & Ei,EO,Ga mee met Otto naar de wigwam. Welke rare sni...,Ga mee met Otto naar de wigwam. Welke rare sni...,,21 mrt 2017 12:15 - Seizoen 11 Afl. 2 - De wig...,https://images.npo.nl/header/2560x1440/878869.jpg,false,https://www.npostart.nl/VPWON_1272823,"(Ga, mee, met, Otto, naar, de, wigwam, ., Welk...",[],[]
